In [17]:
import numpy as np
import pandas as pd
from sklearn import metrics
import heapq


In [18]:

new_dict = dict()
cosine_similarity_list = []
similar_users_list = []


user_cols = ['user_id',
             'age',
             'gender',
             'occupation',
             'zip_code']
users = pd.read_csv('ml-100k/u.user',
                    sep='|',
                    names=user_cols)



In [19]:
users.head()

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [20]:
rating_cols = ['user_id',
               'movie_id',
               'rating',
               'timestamp']
ratings = pd.read_csv('ml-100k/u.data',
                      sep='\t',
                      names=rating_cols)



In [21]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [22]:
item_cols = ['movie id',
             'movie title',
             'release date',
             'video release date',
             'IMDb URL',
             'Unknown',
             'Action',
             'Adventure',
             'Animation',
             'Childrens',
             'Comedy',
             'Crime',
             'Documentary',
             'Drama',
             'Fantasy',
             'FilmNoir',
             'Horror',
             'Musical',
             'Mystery',
             'Romance',
             'SciFi',
             'Thriller',
             'War',
             'Western']
items = pd.read_csv('ml-100k/u.item',
                    sep='|',
                    names=item_cols,
                    encoding='latin-1')



In [23]:
items.head()

,movie id,movie title,release date,video release date,IMDb URL,Unknown,Action,Adventure,Animation,Childrens,...,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [24]:
utility = ratings.pivot(index='user_id',
                        columns='movie_id',
                        values='rating')

utility.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
user_means = utility.mean(axis=1)
user_means.head()

user_id
1    3.610294
2    3.709677
3    2.796296
4    4.333333
5    2.874286
dtype: float64

In [26]:

utility_centered = utility - user_means
utility_centered = utility_centered.where((pd.notnull(utility_centered)), 0)

print("Utility")

print(utility_centered)

Utility
             1         2         3         4         5         6         7     \
user_id                                                                         
1        1.389706 -0.709677  1.203704 -1.333333  0.125714  1.364929  0.034739   
2        0.389706  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
5        0.389706 -0.709677  0.000000  0.000000  0.000000  0.000000  0.000000   
6        0.389706  0.000000  0.000000  0.000000  0.000000  0.000000 -1.965261   
7        0.000000  0.000000  0.000000  0.666667  0.000000  0.000000  1.034739   
8        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000 -0.965261   
9        0.000000  0.000000  0.000000  0.000000  0.000000  1.364929  0.034739   
10       0.389706  0.000000  0.000000 -0.333333  0.000000  0.000000  0.034739   
11       0.000000  0

In [27]:
user_1 = utility_centered[0:1]

for index, row in utility_centered.iterrows():
    np_row = np.array(row)
    np_row.resize(1, 1682)
    if index != 1:
        list = metrics.pairwise.cosine_similarity(user_1, np_row)
        cosine_similarity_list.append(list)
        new_dict.__setitem__(index, list)

similar_users_top_ten = heapq.nlargest(10, new_dict, key=new_dict.get)
user_columns = ['user_id' , '508']

data =0
for i in similar_users_top_ten:

        print("Item: ", i); print("Cosine Similarity ", new_dict.get(i));
        similar_users_list.append(utility_centered.loc[i, 508])

Item:  738
Cosine Similarity  [[ 0.29148679]]
Item:  592
Cosine Similarity  [[ 0.27840172]]
Item:  276
Cosine Similarity  [[ 0.26815054]]
Item:  267
Cosine Similarity  [[ 0.26476147]]
Item:  643
Cosine Similarity  [[ 0.2640026]]
Item:  757
Cosine Similarity  [[ 0.26236785]]
Item:  457
Cosine Similarity  [[ 0.26233704]]
Item:  606
Cosine Similarity  [[ 0.26084701]]
Item:  916
Cosine Similarity  [[ 0.25562438]]
Item:  44
Cosine Similarity  [[ 0.2529544]]


In [28]:
np_array_similar_users = np.array(similar_users_list)

print("Expected rating of user 1 for item 508:")
print(np_array_similar_users.mean())

Expected rating of user 1 for item 508:
0.268965517241
